In [ ]:
import cv2
import mediapipe as mp
import socket
mp_drawing = mp.solutions.drawing_utils #初始化了MediaPipe 的绘图工具包，包括绘制关键点，绘制连接线 
mp_drawing_styles = mp.solutions.drawing_styles #初始化了绘图样式工具包，包括姿态检测样式，手部检测样式，面部检测样式 
mp_pose = mp.solutions.pose #初始化了 MediaPipe 的姿态检测模型，包括人体姿态检测，33 个关键点，世界坐标，归一化坐标 
TCP_IP = "127.0.0.1" # 指定目标设备的 IP 地址，在开发和测试阶段，用于在同一台计算机上运行的 Python 程序和 Unity 应用之间通信 
TCP_PORT = 12000 #指定目标端口号，确保数据被发送到 Unity 应用监听的特定端口 
MESSAGE = b"Hello, Unity Again!"

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue


    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    try:
      Nose=results.pose_world_landmarks.landmark[0]
      LeftWrist=results.pose_world_landmarks.landmark[15]
      RightWrist=results.pose_world_landmarks.landmark[16]
      LeftIndex=results.pose_world_landmarks.landmark[19]
      RightIndex=results.pose_world_landmarks.landmark[20]
      MSGPose=[Nose, LeftWrist, RightWrist, LeftIndex, RightIndex]
      PoseString=''
      for joint in MSGPose:
        PoseString=PoseString+str(joint.x)+','+str(joint.y)+','+str(joint.z)+';'
      sock = socket.socket()
      sock.connect((TCP_IP, TCP_PORT))
      sock.send(PoseString.encode('utf-8'))
      sock.close()
    except:
      print('An error occured.')
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
sock.close()
cv2.destroyAllWindows()

An error occured.
An error occured.
An error occured.
An error occured.
An error occured.
An error occured.
An error occured.
An error occured.
An error occured.
An error occured.
An error occured.
An error occured.


In [2]:
# import string
mystring=str(1.0)+','+str(2.0)+','+str(3.0)+';'+str(4.0)+','+str(5.0)+','+str(6.0)
print(mystring.encode('utf-8'))
# print(string.encode(str))

b'1.0,2.0,3.0;4.0,5.0,6.0'
